**Correzione Verifica Neo4j 26/11/2021 - Pierro**

In [ ]:
!pip install neo4j-driver

     |████████████████████████████████| 89 kB 4.0 MB/s 
  Created wheel for neo4j-driver: filename=neo4j_driver-4.4.0-py3-none-any.whl size=114976 sha256=6f1090faaf7d1983e66959a62b0aca891a91040e59e6938c1e48cadcce8a228e
  Stored in directory: /root/.cache/pip/wheels/5e/89/23/48d22f30bfad0866aac1078fb2e24db1dd3750c46198557ffe
Successfully built neo4j-driver


In [ ]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd

In [ ]:
driver = GraphDatabase.driver("bolt://3.238.63.58:7687", auth=basic_auth("neo4j", "total-arrangements-posts"))
def read_cypher(query):
  with driver.session(database="neo4j") as session:
    results = session.run(query).data()
    df = pd.DataFrame(results)
    col_names = df.columns.values.tolist()
    for c in col_names:
        if type(df[c].iloc[0]) is dict:
          df = pd.concat([df, df[c].apply(pd.Series)], axis=1)
          df.drop(columns=c, inplace=True)
        else:
          df1 = df.pop(c)
          df[c] = df1
    return df

Prendere in considerazione su Neo4j sandbox il database “Crime Investigation” ed utilizzare il comando

```
call db.schema.visualization()
```

per avere un quadro dei nodi e degli archi che lo compongono.

Realizzare poi un notebook Python che permetta di estrarre le seguenti informazioni:

In [ ]:
# 1. Il numero di crimini presenti nel database
cypher_query = '''
MATCH (c:Crime)
RETURN COUNT(c) AS total
'''

read_cypher(cypher_query)

,total
0,28762


In [ ]:
# 2. i crimini che hanno coinvolto le auto ('Vehicle crime')
cypher_query = '''
MATCH (c:Crime {type: 'Vehicle crime'})
RETURN c
'''

read_cypher(cypher_query)

,date,id,type,last_outcome
0,1/08/2017,ab71bb05a5b0620dfec3779eead19d2c121cb73ee58e51...,Vehicle crime,Investigation complete; no suspect identified
1,22/08/2017,e35c34d58fa35f2110b059882a34703d132552a4ad408e...,Vehicle crime,Investigation complete; no suspect identified
2,3/08/2017,576f4e50b1cbeb488c89bd28bd1697a1ad48fa0a5f4df8...,Vehicle crime,Investigation complete; no suspect identified
3,19/08/2017,9cdadc1c345c17336210079096c372a7faaae0268e4725...,Vehicle crime,Under investigation
4,1/08/2017,87b7f4fb4e9168b8c46dc5195ae75b68b2c2aeae6d7c36...,Vehicle crime,Investigation complete; no suspect identified
...,...,...,...,...
2593,4/08/2017,6c97609682f9f7e763b08889ec5f222489db1168277320...,Vehicle crime,Investigation complete; no suspect identified
2594,5/08/2017,052fd091670cf8ce4bc781d26b123bde3f9473f2071b4c...,Vehicle crime,Awaiting court outcome
2595,28/08/2017,df8f71aa71671185eaf45dfb1a653a21dd601d4f530eba...,Vehicle crime,Investigation complete; no suspect identified
2596,10/08/2017,d15a6b9e00d1028171cb0ee75efaebf4af8bd46df0eae7...,Vehicle crime,Investigation complete; no suspect identified


In [ ]:
# 3. Il numero di crimini per ogni tipo di crimine
cypher_query = '''
MATCH (c:Crime)
RETURN c.type AS type, COUNT(c) AS total
'''

read_cypher(cypher_query)

,type,total
0,Public order,4839
1,Robbery,541
2,Theft from the person,423
3,Vehicle crime,2598
4,Burglary,2807
5,Criminal damage and arson,3587
6,Violence and sexual offences,8765
7,Other theft,2140
8,Shoplifting,1427
9,Possession of weapons,236


In [ ]:
# 4. Le tipologie di crimine con più di 2000 denunce
cypher_query = '''
MATCH (c:Crime)
WITH c.type AS type, COUNT(c) AS total
WHERE total > 2000
RETURN type, total
'''

read_cypher(cypher_query)

,type,total
0,Public order,4839
1,Vehicle crime,2598
2,Burglary,2807
3,Criminal damage and arson,3587
4,Violence and sexual offences,8765
5,Other theft,2140


In [ ]:
# 5. I crimini avvenuti in '197 Mason Street'
cypher_query = '''
MATCH (c:Crime)-[:OCCURRED_AT]-(:Location {address: '197 Mason Street'})
RETURN c.type AS type
'''

read_cypher(cypher_query)

,type
0,Violence and sexual offences
1,Burglary


In [ ]:
# 6. Le persone coinvolte nei crimini avvenuti in '197 Mason Street'
cypher_query = '''
MATCH (p:Person)-[:PARTY_TO]-(:Crime)-[:OCCURRED_AT]-(:Location {address: '197 Mason Street'})
RETURN p
'''

read_cypher(cypher_query)

,name,nhs_no,surname
0,Annie,575-05-6519,George
1,Gary,679-81-5309,Vasquez


In [ ]:
# 7. Le generalità e i numeri di telefono delle persone coinvolte nei crimini avvenuti in '197 Mason Street'
cypher_query = '''
MATCH (ph:Phone)-[:HAS_PHONE]-(p:Person)-[:PARTY_TO]-(:Crime)-[:OCCURRED_AT]-(:Location {address: '197 Mason Street'})
RETURN p.surname AS surname, p.name AS name, ph.phoneNo AS phone
'''

read_cypher(cypher_query)

,surname,name,phone
0,George,Annie,4-(223)417-6164
1,Vasquez,Gary,2-(111)124-5323


In [ ]:
# 8. Tutte le persone conosciute dalle persone coinvolte nei crimini avvenuti in '197 Mason Street'
cypher_query = '''
MATCH (p:Person)<-[:KNOWS]-(:Person)-[:PARTY_TO]-(c:Crime)-[:OCCURRED_AT]-(l:Location {address: '197 Mason Street'})
RETURN p
'''

read_cypher(cypher_query)

,name,nhs_no,surname
0,Matthew,569-73-6017,Phillips
1,Rachel,568-22-7686,Hunter
2,Carl,271-53-9609,Lawrence
3,Ashley,554-93-4466,Robertson
4,Lois,691-51-9612,Larson
5,Justin,438-49-1893,Payne
6,Virginia,314-45-9119,Gibson
7,Rebecca,450-68-4090,Lee
8,Nicholas,506-14-4016,Mason
9,Ashley,554-93-4466,Robertson


In [ ]:
# 9. Tutte le persone che sono state chiamate dai malviventi coinvolti nei crimini avvenuti in '197 Mason Street'. Visualizzare anche la data della telefonata
cypher_query = '''
MATCH (p:Person)-[:HAS_PHONE]->(ph:Phone)<-[:CALLED]->(pc:PhoneCall)-[:CALLER]->(:Phone)-[:HAS_PHONE]-(:Person)-[:PARTY_TO]-(:Crime)-[:OCCURRED_AT]-(:Location {address: '197 Mason Street'})
RETURN p, pc.call_date AS date
'''

read_cypher(cypher_query)

,name,nhs_no,surname,date
0,Rachel,568-22-7686,Hunter,20/08/2017
1,Rachel,568-22-7686,Hunter,09/08/2017
2,Ashley,554-93-4466,Robertson,27/08/2017
3,Ashley,554-93-4466,Robertson,01/08/2017
